# Proyecto 2 

github: [link aqui](https://github.com/DanielRasho/MD-Proyecto2)

## Imports

In [2]:
# Core Libraries
import random

# Data Manipulation
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical Analysis
import scipy.stats as stats
import statsmodels.api as sm
from scipy.stats import normaltest

# Machine Learning - Scikit-learn
from sklearn import datasets, metrics, tree
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.compose import ColumnTransformer, make_column_selector as selector
from sklearn.metrics import accuracy_score ,precision_score,recall_score,f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.metrics import (
    confusion_matrix, explained_variance_score, mean_absolute_error, ConfusionMatrixDisplay,
    mean_squared_error, mean_squared_log_error, median_absolute_error, r2_score,accuracy_score, precision_score, recall_score, f1_score, classification_report
)
from sklearn.metrics import root_mean_squared_error
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import (
    GridSearchCV, ShuffleSplit, cross_validate, train_test_split
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree

# Model Evaluation & Utilities
import setuptools.dist
from yellowbrick.regressor import ResidualsPlot

## Cargado de datos

In [3]:
df = pd.read_csv('./data/train.csv')

df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Limpieza de datos

Primero, se realiza una descripción del dataset para obtener un resumen estadístico de los datos. Esto nos permite identificar la distribución de las variables, sus valores mínimos y máximos, la media, la mediana y la desviación estándar. Además, nos ayuda a detectar posibles valores atípicos y comprender mejor la escala de los datos antes de realizar cualquier limpieza o transformación.

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Id,1460.0,730.500000,421.610009,1.0,365.75,730.5,1095.25,1460.0
MSSubClass,1460.0,56.897260,42.300571,20.0,20.00,50.0,70.00,190.0
LotFrontage,1201.0,70.049958,24.284752,21.0,59.00,69.0,80.00,313.0
LotArea,1460.0,10516.828082,9981.264932,1300.0,7553.50,9478.5,11601.50,215245.0
OverallQual,1460.0,6.099315,1.382997,1.0,5.00,6.0,7.00,10.0
OverallCond,1460.0,5.575342,1.112799,1.0,5.00,5.0,6.00,9.0
YearBuilt,1460.0,1971.267808,30.202904,1872.0,1954.00,1973.0,2000.00,2010.0
YearRemodAdd,1460.0,1984.865753,20.645407,1950.0,1967.00,1994.0,2004.00,2010.0
MasVnrArea,1452.0,103.685262,181.066207,0.0,0.00,0.0,166.00,1600.0
BsmtFinSF1,1460.0,443.639726,456.098091,0.0,0.00,383.5,712.25,5644.0


In [5]:
NAValues = [(col, count, (count / len(df)) * 100) for col, count in df.isnull().sum().items() if count > 0]

NAVariablesStats = pd.DataFrame(NAValues, columns=["Variable", "Count", "Percentage"])
NAVariablesStats = NAVariablesStats.sort_values("Count", ascending=False)
NAVariablesStats

,Variable,Count,Percentage
16,PoolQC,1453,99.520548
18,MiscFeature,1406,96.301370
1,Alley,1369,93.767123
17,Fence,1179,80.753425
2,MasVnrType,872,59.726027
10,FireplaceQu,690,47.260274
0,LotFrontage,259,17.739726
11,GarageType,81,5.547945
12,GarageYrBlt,81,5.547945
13,GarageFinish,81,5.547945


### Criterios de eliminacion de columnas

Dado que el dataset contiene una gran cantidad de datos, primero realizamos una búsqueda para identificar las columnas con al menos un valor nulo. Posteriormente, analizamos el porcentaje de valores faltantes en cada una de estas columnas. Si el porcentaje de datos nulos era significativo (considerado alto según nuestro criterio), decidimos eliminarlas para evitar sesgos en el análisis y mejorar la calidad de los datos.

### Columas eliminadas
Las siguientes columnas fueron eliminadas debido a su alto porcentaje de valores nulos, tanto en training como testing:

* PoolQC → 99.52% de datos nulos.
* MiscFeature → 96.30% de datos nulos.
* Alley → 93.77% de datos nulos.
* Fence → 80.75% de datos nulos.
* MasVnrType → 59.73% de datos nulos.
* FireplaceQu → 47.26% de datos nulos.
* GarageQual → 5.55% de datos nulos.
* Id -> Es una variable insignificante que no nos aporta nada.

### Posteriormente

Al ya no existir columnas con un alto numero de valores nulos, ya nos podemos deshacer de las filas que estan incompletas.

In [6]:
# Quitar las columnas con mayor porcentaje de nulos
df.drop(['Id', "Alley", "MasVnrType", "FireplaceQu", "GarageQual", "PoolQC", "Fence", "MiscFeature" ], axis=1, inplace=True)
# Ahora ya se pueden quitar las filas con NA porque son muy pocos
df_cleaned = df.dropna()

# Asegurar que no hayan valores nulos
NAValues = list(df_cleaned.isnull().sum() / len(df_cleaned))
list(value for value in NAValues if value > 0)

[]

Después de obtener un dataset más limpio, el siguiente paso fue analizar su estructura con mayor detalle. Para ello, verificamos la cantidad de filas y columnas restantes, lo que nos permitió entender la dimensión de los datos después del proceso de limpieza.

Además, realizamos una nueva inspección en busca de datos duplicados, junto con los tipos de datos para tener una mejor compresion y asi para evitar cualquier tipo de sesgo.

## Naive Bayes

### Hacer la division estratificada

In [7]:
df_cleaned["SalePriceCategory"] = pd.qcut(df_cleaned["SalePrice"], q=3, labels=["Económico", "Intermedio", "Caro"])
df_cleaned["SalePriceCategory"] = df_cleaned["SalePriceCategory"].astype("category")
train_set, test_set = train_test_split(df_cleaned, test_size=0.2, random_state=42, stratify=df_cleaned["SalePriceCategory"])

/tmp/ipykernel_530105/2724778207.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["SalePriceCategory"] = pd.qcut(df_cleaned["SalePrice"], q=3, labels=["Económico", "Intermedio", "Caro"])
/tmp/ipykernel_530105/2724778207.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned["SalePriceCategory"] = df_cleaned["SalePriceCategory"].astype("category")


## Modelo de Regresion
Se empezára usando un modelo de regresión con Naive Bayes. El primer paso para ello es preprocesar los datos categóricos y númericos por separado.

In [8]:
train_salePrice = train_set["SalePrice"]
test_salePrice = test_set["SalePrice"]
x_train = train_set.drop(columns=["SalePriceCategory", "SalePrice"])
x_test = test_set.drop(columns=["SalePriceCategory", "SalePrice"])

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(x_train)
categorical_columns = categorical_columns_selector(x_train)

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
numerical_preprocessor = StandardScaler()
preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standar-scaler', numerical_preprocessor,numerical_columns)
    ])

A continuación se define el pipeline de scikit-learn que seguiran los datos: 

In [9]:
pipeline = Pipeline(
    [('preprocessor',preprocessor),
     ('regressor',GaussianNB())])

A continuación se configura el modelo con los datos de entrenamiento, que fueron separados en partes anteriores:

In [10]:
print(train_salePrice.shape)
print(x_train.shape)
modelo = pipeline.fit(x_train, train_salePrice)
y_pred = modelo.predict(x_test)

explained_variance = explained_variance_score(test_salePrice, y_pred)
r2 = r2_score(test_salePrice, y_pred)
mae = mean_absolute_error(test_salePrice, y_pred)
mse = mean_squared_error(test_salePrice, y_pred)
rmse = np.sqrt(mse)

# Store the metrics in a DataFrame
metrics_df = pd.DataFrame({
    'Model': ['Naive Bayes Regresión'],
    'Explained Variance': [explained_variance],
    'R2': [r2],
    'MAE': [mae],
    'MSE': [mse],
    'RMSE': [rmse]
})

metrics_df

(875,)
(875, 72)


,Model,Explained Variance,R2,MAE,MSE,RMSE
0,Naive Bayes Regresión,0.362955,0.3285,39495.607306,5.226752e+09,72296.276653


Los resultados no fueron prometedores, y son superados por mucho por modelos de regresion lineal o árboles de decisión. Por lo que en un intento para mejorar su rendimiento se usará validación cruzada conjutn con una variación de parámetros para encontrar aquellos que minimizan el error cuadrático medio.

In [12]:
params = [{
    "regressor__var_smoothing": np.logspace(0, -9, num=100)
}]

modelo = GridSearchCV(pipeline, param_grid=params, n_jobs=2, cv=5, scoring="neg_root_mean_squared_error")
modelo.fit(x_train, train_salePrice)  # Ensure variable names match

y_pred = modelo.predict(x_test)

explained_variance = explained_variance_score(test_salePrice, y_pred)
r2 = r2_score(test_salePrice, y_pred)
mae = mean_absolute_error(test_salePrice, y_pred)
mse = mean_squared_error(test_salePrice, y_pred)
rmse = np.sqrt(mse)

# Store the metrics in a DataFrame
metrics_df = pd.DataFrame({
    'Model': ['Naive Bayes Regresión'],
    'Explained Variance': [explained_variance],
    'R2': [r2],
    'MAE': [mae],
    'MSE': [mse],
    'RMSE': [rmse]
})

metrics_df

/home/smaugthur/Documents/MD-Proyecto2/.env/lib/python3.13/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


,Model,Explained Variance,R2,MAE,MSE,RMSE
0,Naive Bayes Regresión,0.777206,0.770486,27465.680365,1.786468e+09,42266.628227


Los resultados incrementaron en enorme médida, haciendo que el modelo tenga un rendimiento ligeramente mejor al indicado por modelos de regresión lineal ($R^2=0.71$) al lograr un $R^2=0.77$, además tambien se puede interpretar que de acuerdo al RMSE el algoritmo se desvia en promedio $42K del resultado esperado, o $27K de acuerdo al MAE. Lo que no son cifras especialmente significativas sabiendo que hay casas con costos cercanos a $100K

## Modelo de Clasificacion

In [ ]:
train_salePrice = train_set.pop("SalePrice")
test_salePrice = test_set.pop("SalePrice")
y_train = train_set.pop("SalePriceCategory")
y_test = test_set.pop("SalePriceCategory")
x_train = train_set
x_test = test_set

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(train_set)
categorical_columns = categorical_columns_selector(train_set)
print(categorical_columns)
print(numerical_columns)

['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'GarageType', 'GarageFinish', 'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition']
['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']


In [ ]:
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()
preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    # ('standar-scaler', numerical_preprocessor,numerical_columns)
    ])

In [ ]:
pipeline = Pipeline(
    [('preprocessor',preprocessor),
     ('clasifier',MultinomialNB())])

pipeline.get_params()

{'memory': None,
 'steps': [('preprocessor',
   ColumnTransformer(transformers=[('one-hot-encoder',
                                    OneHotEncoder(handle_unknown='ignore'),
                                    ['MSZoning', 'Street', 'LotShape',
                                     'LandContour', 'Utilities', 'LotConfig',
                                     'LandSlope', 'Neighborhood', 'Condition1',
                                     'Condition2', 'BldgType', 'HouseStyle',
                                     'RoofStyle', 'RoofMatl', 'Exterior1st',
                                     'Exterior2nd', 'ExterQual', 'ExterCond',
                                     'Foundation', 'BsmtQual', 'BsmtCond',
                                     'BsmtExposure', 'BsmtFinType1',
                                     'BsmtFinType2', 'Heating', 'HeatingQC',
                                     'CentralAir', 'Electrical', 'KitchenQual',
                                     'Functional', ...])])),
 

Antes de poder entrenar el modelo, los datos necesitan una transformación para que todas las variables se puedan procesar de forma correcta. Se usa one hot encoder para pasar las variables categoricas en una representacion numerica. El pipeline es el encargado de primero procesar los datos y luego entrenar o predecir  el modelo.

In [ ]:
modelo = pipeline.fit(x_train,y_train)

In [ ]:
y_pred = modelo.predict(x_test)
cm = confusion_matrix(y_test,y_pred)
cm
accuracy = accuracy_score(y_test, y_pred)  # Exactitud
precision = precision_score(y_test, y_pred, average='weighted')  # Precisión (weighted para clases desbalanceadas)
recall = recall_score(y_test, y_pred, average='weighted')  # Sensibilidad
f1 = f1_score(y_test, y_pred, average='weighted')  # F1-score

# Mostrar métricas
print(f"Precisión (Accuracy): {accuracy:.4f}")
print(f"Precisión (Precision): {precision:.4f}")
print(f"Sensibilidad (Recall): {recall:.4f}")
print(f"Puntaje F1 (F1-score): {f1:.4f}")

Precisión (Accuracy): 0.6758
Precisión (Precision): 0.6693
Sensibilidad (Recall): 0.6758
Puntaje F1 (F1-score): 0.6702


Como se puede observar el modelo clasifica correctamente (accuracy) el 67.58% de las veces, esto puede deberse a que los datos esten desbalanceados entre categorias como podria ser el caso de la categoria "caro" el cual no contiene tatos datos. 

En cuanto a las demas metricas obtenidas se puede concluir que el modelo esta equilibrado pero si a futuro se llegara a necesat un modelo que requiera una alta precision o sensibilidad se podrias hacer tunning a los datos ingresados para construir el modelo.

In [ ]:
# Mostrar reporte de clasificación
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))
print("\nMatriz de Clasificación:")
print(cm)


Reporte de Clasificación:
              precision    recall  f1-score   support

        Caro       0.78      0.78      0.78        73
   Económico       0.68      0.78      0.73        73
  Intermedio       0.55      0.47      0.50        73

    accuracy                           0.68       219
   macro avg       0.67      0.68      0.67       219
weighted avg       0.67      0.68      0.67       219


Matriz de Clasificación:
[[57  4 12]
 [ 0 57 16]
 [16 23 34]]


En base a estos resultados podemos observar que donde tuvo mas problemas el modelo para poder detectar clasificaciones fue en el aparatado de "intermedio". Teniendo una precision de 0.55 y recall de 0.47, el modelo se esta confundiendo con las otras clases teniendo un total de 50 clasificaciones erroneas.